# Testing Nsaba Functionality

##### Current Methods
- est_aba_ge()
- ge_ratio()
- est_ns_act()
- make_ge_ns_mat()

NsabaVisualizer methods also tested.

In [1]:
from nsaba.nsaba import Nsaba
from nsaba.nsaba.visualizer import NsabaVisualizer
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import itertools
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
%load_ext line_profiler

In [3]:
# Simon Path IO

data_dir = '../../data_dir'
os.chdir(data_dir)
Nsaba.aba_load()
Nsaba.ns_load()

This may take a minute or two ...
SampleAnnot.csv loaded.
MicroarrayExpression.csv loaded.
Probes.csv loaded.
Nsaba.aba['mni_coords'] initialized.

This may take a minute or two ...
database.txt loaded.
features.txt loaded.
Nsaba.ns['mni_coords'] initialized.



In [6]:
# Loading gene expression for all ABA registered Entrez IDs.
A = Nsaba()
# A.load_ge_pickle('Nsaba_ABA_ge.pkl')

In [24]:
entrids = [1813,1816]
coords = [(np.random.uniform(-50,50), np.random.uniform(-50,50), np.random.uniform(-50,50)) for _ in range(50)]
A.ge_ratio(entrids, np.array(coords), rnn_args={'radius':10})

array([        nan,  0.46101267,         nan,         nan,         nan,
        1.29863454,  0.46767585,  0.54803596,  0.5424098 ,  0.6102441 ,
               nan,  1.25713839,  0.50261043,  0.72005311,  0.61401404,
        0.55603219,  0.53315377,         nan,         nan,         nan,
        0.47466528,  0.57446139,  0.51203572,  0.74924179,  1.13694337,
               nan,         nan,         nan,  0.518115  ,  0.46028177,
        0.59801775,  0.74915265,  0.59348125,         nan,  0.64046317,
               nan,  0.49174621,         nan,         nan,  0.54735262,
               nan,         nan,  0.61095538,         nan,         nan,
               nan,  0.49135148,  0.51203572,  0.52147863,  1.08939184])

In [25]:
entrids = [1813,1816]
A.estimate_aba_ge(entrids, coords=coords, radius=5)

In [26]:
A.ge[1813][1030282]

{'GE': array([        nan,         nan,         nan,         nan,         nan,
                nan,         nan,  3.30760913,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,  2.02509563,         nan,
                nan,  4.02299513,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan,  2.76059745,         nan,         nan,
                nan,         nan,         nan,         nan,         nan]),
 'classifer': RadiusNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
              metric_params=None, p=2, radius=5, weights='uniform')}

In [30]:
A.ge[1813]["mean"]

{'GE': array([        nan,         nan,         nan,         nan,         nan,
                nan,         nan,  3.40855287,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,  3.10039167,         nan,
                nan,  5.74378313,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan,  3.36705027,         nan,         nan,
                nan,         nan,         nan,         nan,         nan]),
 'classifer': RadiusNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
              metric_params=None, p=2, radius=5, weights='uniform')}

In [ ]:
%time A.estimate_ns_act('attention')
A.estimate_ns_act('reward')

## Visualization Methods (testing)

In [32]:
NV = NsabaVisualizer(A)

In [33]:
A.ge[1813]["mean"]["GE"]

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,  3.40855287,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,  3.10039167,         nan,
               nan,  5.74378313,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,  3.36705027,         nan,         nan,
               nan,         nan,         nan,         nan,         nan])

In [ ]:
%debug NV.visualize_ge([1813])

NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> <string>(1)<module>()

ipdb> s
--Call--
> /Users/simonhaxby/Code/Python/nsaba/nsaba/visualizer.py(24)visualize_ge()
     23 
---> 24     def visualize_ge(self, gene, alpha=0.4):
     25         """

ipdb> n
> /Users/simonhaxby/Code/Python/nsaba/nsaba/visualizer.py(36)visualize_ge()
     35         """
---> 36         for e in gene:
     37             if e in self.no.ge:

ipdb> n
> /Users/simonhaxby/Code/Python/nsaba/nsaba/visualizer.py(37)visualize_ge()
     36         for e in gene:
---> 37             if e in self.no.ge:
     38                 fig = plt.figure()

ipdb> n
> /Users/simonhaxby/Code/Python/nsaba/nsaba/visualizer.py(38)visualize_ge()
     37             if e in self.no.ge:
---> 38                 fig = plt.figure()
     39                 ax = fig.add_subplot(111, projection='3d')

ipdb> p e
1813
ipdb> n
> /Users/simonhaxby/Code/Python/nsaba/nsaba/visualizer.py(39)visualize_ge()
     38                 fig = 

In [ ]:
NV.visualize_ns('attention', alpha=.3)

In [ ]:
NV.lstsq_ns_ge('attention', [1813]);

In [ ]:
NV.lstsq_ge_ge(1813, 1816);

In [ ]:
NV.lstsq_ns_ns('attention', 'reward');

In [ ]:
A.matrix_builder(['reward'], [1813,1816])